In [1]:
# Import libraries

# To write a Python 2/3 compatible codebase, the first step is to add this line to the top of each module
from __future__ import division, print_function, unicode_literals

import pandas as pd
import numpy as np
import scipy as sp
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#import pandas_profiling
import os, sys

import datetime

import numpy

import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler # Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import MinMaxScaler

import gc

gc.collect()

0

In [2]:
# Load datasets

transactions = pd.read_csv("transaction_table_supp.csv")
product = pd.read_csv("product_table_supp.csv")
seasonality = pd.read_csv("seasonality.csv")
holiday = pd.read_csv("holiday.csv")
promo = pd.read_csv("promo_ad.csv")

total_product = pd.read_csv("product_table.csv")
total_transactions = pd.read_csv("transaction_table.csv")

In [3]:
# Set of columns and rows to display

pd.set_option('max_columns', None)
pd.set_option('max_rows', 500)

# Data Exploration

In [4]:
total_product.head()

,prod_id,subcategory_id,sub_category_desc,category_id,category_desc,brand_desc,category_desc_eng
0,145519008,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH
1,145519009,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH
2,145519010,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH
3,145519011,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH
4,145519012,93970,IOGURTE BIFIDUS LIQ,95854,IOGURTE SAUDE,ACTIVIA,YOGURT HEALTH


In [5]:
# Category level transactional statistics - Beer in Pernalonga stores

total_product = total_product[total_product['category_desc_eng'] == "BEER WITH ALCOHOL"]

beer_idlist = total_product['prod_id']

In [6]:
# Time period for total transactions is aligned with time period for transactions 

total_transactions = total_transactions[total_transactions.prod_id.isin(beer_idlist)]

In [7]:
total_transactions['beer_sale'] = sum(total_transactions.tran_prod_sale_amt)
total_transactions['beer_net_sale'] = sum(total_transactions.tran_prod_paid_amt)
total_transactions['beer_sale_qty'] = sum(total_transactions.tran_prod_sale_qty)
total_transactions['beer_avg_discount_amt'] = sum(abs(total_transactions.tran_prod_discount_amt)) / sum(total_transactions.tran_prod_sale_qty)

In [8]:
# Unique values

total_transactions = total_transactions[['beer_sale', 'beer_net_sale', 'beer_sale_qty', 'beer_avg_discount_amt']].drop_duplicates().reset_index(drop=True)

In [9]:
# Export to CSV

total_transactions.to_csv('category_summary.csv', index=False)

In [10]:
# Product level transactional statistics - For each beer product in Pernalonga stores

transactions['mean_unit_price'] = transactions.groupby('prod_id').prod_unit_price.transform('mean')
transactions['median_unit_price'] = transactions.groupby('prod_id').prod_unit_price.transform('median')
transactions['min_unit_price'] = transactions.groupby('prod_id').prod_unit_price.transform('min')
transactions['max_unit_price'] = transactions.groupby('prod_id').prod_unit_price.transform('max')
transactions['total_quantity_sold'] = transactions.groupby('prod_id').tran_prod_sale_qty.transform('sum')
transactions['product_total_sales'] = transactions.groupby('prod_id').tran_prod_sale_amt.transform('sum')
transactions['product_total_net_sales'] = transactions.groupby('prod_id').tran_prod_paid_amt.transform('sum')
transactions['avg_discount_amt'] = transactions.groupby('prod_id').tran_prod_discount_amt.transform('sum') / transactions.groupby('prod_id').tran_prod_sale_qty.transform('sum')
transactions['pct_sales_category'] = transactions.product_total_net_sales.apply(lambda x: x/total_transactions.beer_sale)
transactions['pct_net_sales_category'] = transactions.product_total_net_sales.apply(lambda x: x/total_transactions.beer_net_sale)
transactions['pct_sales_qty_category'] = transactions.product_total_net_sales.apply(lambda x: x/total_transactions.beer_sale_qty)

In [11]:
transactions.head()

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price,mean_unit_price,median_unit_price,min_unit_price,max_unit_price,total_quantity_sold,product_total_sales,product_total_net_sales,avg_discount_amt,pct_sales_category,pct_net_sales_category,pct_sales_qty_category
0,14749832,2.016060e+18,2016-06-28,999,138936953,CT,44.97,3,0.0,0,44.97,14.99,15.631935,14.99,14.99,16.99,1261,19707.39,18107.39,1.268834,0.010784,0.016588,0.021212
1,6019835,2.016070e+18,2016-07-21,999,138936952,CT,7.98,2,0.0,0,7.98,3.99,4.282945,4.29,3.99,4.59,10895,46670.55,44402.05,0.208215,0.026443,0.040677,0.052015
2,18829966,2.016070e+18,2016-07-30,999,138936952,CT,19.95,5,2.5,1,17.45,3.99,4.282945,4.29,3.99,4.59,10895,46670.55,44402.05,0.208215,0.026443,0.040677,0.052015
3,18829966,2.016100e+18,2016-10-30,999,138936951,CT,1.50,2,0.1,1,1.40,0.75,0.745747,0.75,0.69,0.79,22477,16767.35,15999.25,0.034173,0.009528,0.014657,0.018742
4,45089965,2.016120e+18,2016-12-12,999,138936951,CT,2.25,3,0.0,0,2.25,0.75,0.745747,0.75,0.69,0.79,22477,16767.35,15999.25,0.034173,0.009528,0.014657,0.018742


In [12]:
# Unique values

transactions_a = transactions[['prod_id', 'mean_unit_price', 'median_unit_price', 'min_unit_price', 'max_unit_price',
                                     'total_quantity_sold', 'product_total_sales', 'product_total_net_sales','avg_discount_amt', 
                                     'pct_sales_category', 'pct_net_sales_category', 'pct_sales_qty_category']].drop_duplicates().reset_index(drop=True)

In [13]:
# Export to CSV

transactions_a.to_csv('product_summary.csv', index=False)

In [14]:
promo.head()

,tran_wk,vehicle,amount,unit,prod_assoc
0,2015-12-27,Flyer,1,NaN,138936952
1,2015-12-27,Flyer,1,NaN,138936953
2,2015-12-27,Paid Search,19444,impressions,ALL
3,2016-01-03,Paid Search,47733,impressions,ALL
4,2016-01-10,Paid Search,29342,impressions,ALL


In [15]:
promo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tran_wk     188 non-null    object
 1   vehicle     188 non-null    object
 2   amount      188 non-null    int64 
 3   unit        134 non-null    object
 4   prod_assoc  188 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.5+ KB


In [16]:
# Promotion Ads schedule
# First, we want to compute the days difference between Ads to discover how the promotion ads were scheduled.

promo['tran_wk'] = pd.to_datetime(promo.tran_wk, infer_datetime_format=True)

In [17]:
promo.prod_assoc.replace(to_replace='ALL', value = np.nan, inplace=True)

In [18]:
daydifference = pd.DataFrame(promo['tran_wk'].drop_duplicates())

daydifference

,tran_wk
0,2015-12-27
3,2016-01-03
4,2016-01-10
5,2016-01-17
6,2016-01-24
7,2016-01-31
8,2016-02-07
10,2016-02-14
11,2016-02-21
12,2016-02-28


In [19]:
daydifference['lag_wk'] = promo['tran_wk'].drop_duplicates()

daydifference

,tran_wk,lag_wk
0,2015-12-27,2015-12-27
3,2016-01-03,2016-01-03
4,2016-01-10,2016-01-10
5,2016-01-17,2016-01-17
6,2016-01-24,2016-01-24
7,2016-01-31,2016-01-31
8,2016-02-07,2016-02-07
10,2016-02-14,2016-02-14
11,2016-02-21,2016-02-21
12,2016-02-28,2016-02-28


In [20]:
daydifference['lag_wk'] = daydifference['lag_wk'].shift(1)

In [21]:
daydifference['daydiff'] = (daydifference.tran_wk - daydifference.lag_wk).dt.days

daydifference

,tran_wk,lag_wk,daydiff
0,2015-12-27,NaT,NaN
3,2016-01-03,2015-12-27,7.0
4,2016-01-10,2016-01-03,7.0
5,2016-01-17,2016-01-10,7.0
6,2016-01-24,2016-01-17,7.0
7,2016-01-31,2016-01-24,7.0
8,2016-02-07,2016-01-31,7.0
10,2016-02-14,2016-02-07,7.0
11,2016-02-21,2016-02-14,7.0
12,2016-02-28,2016-02-21,7.0


In [22]:
# The time difference between each Ad is 7 days. So, we can confirm that the promotion ads were distributed on a weekly basis. 

daydifference.daydiff.unique()

array([nan,  7.])

In [23]:
# Then, we'd like to learn if the promotion ads were distributed in each week between 2016 and 2017

promo.tran_wk.unique()

array(['2015-12-27T00:00:00.000000000', '2016-01-03T00:00:00.000000000',
       '2016-01-10T00:00:00.000000000', '2016-01-17T00:00:00.000000000',
       '2016-01-24T00:00:00.000000000', '2016-01-31T00:00:00.000000000',
       '2016-02-07T00:00:00.000000000', '2016-02-14T00:00:00.000000000',
       '2016-02-21T00:00:00.000000000', '2016-02-28T00:00:00.000000000',
       '2016-03-06T00:00:00.000000000', '2016-03-13T00:00:00.000000000',
       '2016-03-20T00:00:00.000000000', '2016-03-27T00:00:00.000000000',
       '2016-04-03T00:00:00.000000000', '2016-04-10T00:00:00.000000000',
       '2016-04-17T00:00:00.000000000', '2016-04-24T00:00:00.000000000',
       '2016-05-01T00:00:00.000000000', '2016-05-08T00:00:00.000000000',
       '2016-05-15T00:00:00.000000000', '2016-05-22T00:00:00.000000000',
       '2016-05-29T00:00:00.000000000', '2016-06-05T00:00:00.000000000',
       '2016-06-12T00:00:00.000000000', '2016-06-19T00:00:00.000000000',
       '2016-06-26T00:00:00.000000000', '2016-07-03

In [24]:
len(promo.tran_wk.unique())

106

In [25]:
min_date = min(promo.tran_wk)

min_date

Timestamp('2015-12-27 00:00:00')

In [26]:
max_date = max(promo.tran_wk)

max_date

Timestamp('2017-12-31 00:00:00')

In [27]:
# There are 106 unique tran_wk in promo, which is greater than the 105 weeks time difference between
# the earliest and latest date in the table. Thus, we can confirm the ads were distributed each week. 

def diff(start, end):
    days = pd.to_datetime(end) - pd.to_datetime(start)
    week = int(pd.Timedelta(days).days / 7)
    remainder = pd.Timedelta(days).days % 7
    return str(week) + ' weeks and ' + str(remainder) + ' days'

diff(min_date, max_date)

'105 weeks and 0 days'

# Data Preparation

In [28]:
# Reach For TV & Radio
# Given the GRP and reach function for TV and Radio, we first computed the reach for each vehicle in each week.
# Alpha of TV and Radio
# TV - 8 week half-life, Radio - 4 week half-life 
# calculate the alpha of TV and Radio based on half-life of each vehicle

a_TV = 1 - 0.5**(1/8)
a_Radio = 1 - 0.5**(1/4)

In [29]:
# Adstock & Reach for TV
# Compute weekly adstock and reach for TV. The earlist distribution date of TV ads is 2016-06-05
TV = promo[promo.vehicle == "TV"]
wk = pd.DataFrame(promo['tran_wk'])

TV = wk.merge(TV, on='tran_wk', how='left').drop_duplicates().reset_index()
TV['ad_stock'] = 0
TV['reach'] = 0
TV.fillna(0, inplace=True)
TV = TV.drop("index", axis=1)

TV

,tran_wk,vehicle,amount,unit,prod_assoc,ad_stock,reach
0,2015-12-27,0,0.0,0,0,0,0
1,2016-01-03,0,0.0,0,0,0,0
2,2016-01-10,0,0.0,0,0,0,0
3,2016-01-17,0,0.0,0,0,0,0
4,2016-01-24,0,0.0,0,0,0,0
5,2016-01-31,0,0.0,0,0,0,0
6,2016-02-07,0,0.0,0,0,0,0
7,2016-02-14,0,0.0,0,0,0,0
8,2016-02-21,0,0.0,0,0,0,0
9,2016-02-28,0,0.0,0,0,0,0


In [30]:
# Adstock - Adstock(t) = aGRP(t) + (1-a)Adstock(t-1)
# Reach for TV: Reach = 0.95(1-e^(-0.020GRP))
for i in range(2,106):
  TV.loc[i, 'ad_stock'] = a_TV * TV.loc[i, 'amount'] + (1-a_TV) * TV.loc[i-1, 'ad_stock'] 
  TV.loc[i, 'reach'] = 0.95 * (1 - np.exp(-0.02 * TV.loc[i, 'ad_stock']))
 
 

In [31]:
TV

,tran_wk,vehicle,amount,unit,prod_assoc,ad_stock,reach
0,2015-12-27,0,0.0,0,0,0.000000,0.000000
1,2016-01-03,0,0.0,0,0,0.000000,0.000000
2,2016-01-10,0,0.0,0,0,0.000000,0.000000
3,2016-01-17,0,0.0,0,0,0.000000,0.000000
4,2016-01-24,0,0.0,0,0,0.000000,0.000000
5,2016-01-31,0,0.0,0,0,0.000000,0.000000
6,2016-02-07,0,0.0,0,0,0.000000,0.000000
7,2016-02-14,0,0.0,0,0,0.000000,0.000000
8,2016-02-21,0,0.0,0,0,0.000000,0.000000
9,2016-02-28,0,0.0,0,0,0.000000,0.000000


In [32]:
# Adstock & Reach for Radio
# Compute weekly adstock and reach for Radio. The earlist distribution date of TV ads is 2016-06-05
Radio = promo[promo.vehicle == "Radio"]
wk = pd.DataFrame(promo['tran_wk'])
 
Radio = wk.merge(Radio, on='tran_wk', how='left').drop_duplicates().reset_index()
Radio['ad_stock'] = 0
Radio['reach'] = 0
Radio.fillna(0, inplace=True)
Radio = Radio.drop("index", axis=1)

In [33]:
# Adstock - Adstock(t) = aGRP(t) + (1-a)Adstock(t-1)
# Reach for Radio: Reach = 0.95(1-e^(-0.020GRP))
for i in range(2,106):
  Radio.loc[i, 'ad_stock'] = a_Radio * Radio.loc[i, 'amount'] + (1-a_Radio) * Radio.loc[i-1, 'ad_stock'] 
  Radio.loc[i, 'reach'] = 0.95 * (1 - np.exp(-0.02 * Radio.loc[i, 'ad_stock']))

In [34]:
Radio

,tran_wk,vehicle,amount,unit,prod_assoc,ad_stock,reach
0,2015-12-27,0,0.0,0,0,0.000000,0.000000
1,2016-01-03,0,0.0,0,0,0.000000,0.000000
2,2016-01-10,0,0.0,0,0,0.000000,0.000000
3,2016-01-17,0,0.0,0,0,0.000000,0.000000
4,2016-01-24,0,0.0,0,0,0.000000,0.000000
5,2016-01-31,0,0.0,0,0,0.000000,0.000000
6,2016-02-07,0,0.0,0,0,0.000000,0.000000
7,2016-02-14,0,0.0,0,0,0.000000,0.000000
8,2016-02-21,0,0.0,0,0,0.000000,0.000000
9,2016-02-28,0,0.0,0,0,0.000000,0.000000


In [35]:
# Change column name of reach to "TV" and "Reach", so we can use them as model input variables

TV.rename(columns={"ad_stock": "TV_ad_stock", "reach": "TV" }, inplace=True)
Radio.rename(columns={"ad_stock": "Radio_ad_stock", "reach": "Radio" }, inplace=True)

In [36]:
Radio

,tran_wk,vehicle,amount,unit,prod_assoc,Radio_ad_stock,Radio
0,2015-12-27,0,0.0,0,0,0.000000,0.000000
1,2016-01-03,0,0.0,0,0,0.000000,0.000000
2,2016-01-10,0,0.0,0,0,0.000000,0.000000
3,2016-01-17,0,0.0,0,0,0.000000,0.000000
4,2016-01-24,0,0.0,0,0,0.000000,0.000000
5,2016-01-31,0,0.0,0,0,0.000000,0.000000
6,2016-02-07,0,0.0,0,0,0.000000,0.000000
7,2016-02-14,0,0.0,0,0,0.000000,0.000000
8,2016-02-21,0,0.0,0,0,0.000000,0.000000
9,2016-02-28,0,0.0,0,0,0.000000,0.000000


In [37]:
TV_Radio = TV.merge(Radio, on='tran_wk', how='left').reset_index(drop=True)

In [38]:
TV_Radio

,tran_wk,vehicle_x,amount_x,unit_x,prod_assoc_x,TV_ad_stock,TV,vehicle_y,amount_y,unit_y,prod_assoc_y,Radio_ad_stock,Radio
0,2015-12-27,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
1,2016-01-03,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
2,2016-01-10,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
3,2016-01-17,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
4,2016-01-24,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
5,2016-01-31,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
6,2016-02-07,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
7,2016-02-14,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
8,2016-02-21,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000
9,2016-02-28,0,0.0,0,0,0.000000,0.000000,0,0.0,0,0,0.000000,0.000000


In [39]:
# Weekly Transactional Data
# Change first day of week
# The ad was given every Sunday, so we changed the first day of each week to Sunday to include the sales data
# within 7 days of the ad distribtion date.

transactions['tran_dt'] = pd.to_datetime(transactions['tran_dt'], infer_datetime_format=True)

from pandas.tseries.offsets import Week

transactions['tran_wk'] = transactions['tran_dt'].where(transactions['tran_dt'].dt.weekday == 6, transactions['tran_dt'] - Week(weekday=6))

In [40]:
# Sale Volume and Average Discount

transactions['weekly_sale_qty'] = transactions.groupby(['prod_id', 'tran_wk'] ).tran_prod_sale_qty.transform('sum')
transactions['weekly_avg_disc'] = transactions.groupby(['prod_id', 'tran_wk'] ).tran_prod_paid_amt.transform('sum') / transactions.groupby(['prod_id', 'tran_wk'] ).tran_prod_sale_amt.transform('sum') 
transactions['weekly_avg_disc'] = 1 - transactions['weekly_avg_disc']
transactions['weekly_store_sales'] = transactions.groupby(['prod_id', 'tran_wk', 'store_id'] ).tran_prod_paid_amt.transform('sum')

In [41]:
# Weekly Shelf Price

transactions['weekly_shelf_price'] = transactions.groupby(['prod_id', 'tran_wk'] ).prod_unit_price.transform('mean')

# By checking weekly unit sale price across stores, we found the brand adopted uniform prices for 
# the three products. That said, the unit sale price at a given week is fixed and not impacted by store location.

transactions['weekly_unique_shelf_price'] = transactions.groupby(['prod_id', 'tran_wk'] ).prod_unit_price.transform(lambda x: x.nunique())

In [42]:
max(transactions['weekly_unique_shelf_price'])

1

In [43]:
# Seasonality & Holiday
# Include seasonality factor in the model input data

seasonality['tran_wk'] = pd.to_datetime(seasonality.tran_wk, infer_datetime_format=True)

In [44]:
weekly_data = transactions[['prod_id','weekly_sale_qty','weekly_avg_disc','weekly_shelf_price','tran_wk']]
weekly_data = weekly_data.drop_duplicates().reset_index()

In [45]:
weekly_data = seasonality.merge(weekly_data, how='inner', on='tran_wk' )

In [46]:
weekly_data

,tran_wk,seas_index,index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price
0,2015-12-27,5601.346939,3669,138936951,6,0.144928,0.69
1,2015-12-27,5601.346939,4545,138936952,18,0.125313,3.99
2,2015-12-27,5601.346939,5370,138936953,3,0.133422,14.99
3,2016-01-03,5435.051020,16,138936951,213,0.000000,0.69
4,2016-01-03,5435.051020,17,138936953,3,0.000000,14.99
5,2016-01-03,5435.051020,159,138936952,109,0.000000,3.99
6,2016-01-10,5335.275510,18,138936952,92,0.000000,3.99
7,2016-01-10,5335.275510,441,138936951,183,0.000000,0.69
8,2016-01-10,5335.275510,1904,138936953,6,0.000000,14.99
9,2016-01-17,5262.316327,247,138936952,86,0.000000,3.99


In [47]:
# Include Holiday factor in the model input data - each holiday will be used as a seperate variable in the model

holiday['tran_wk'] = pd.to_datetime(holiday.tran_wk, infer_datetime_format=True)
weekly_data = weekly_data.merge(holiday, how='left', on='tran_wk' )

In [48]:
weekly_data

,tran_wk,seas_index,index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday
0,2015-12-27,5601.346939,3669,138936951,6,0.144928,0.69,NEWYEAR
1,2015-12-27,5601.346939,4545,138936952,18,0.125313,3.99,NEWYEAR
2,2015-12-27,5601.346939,5370,138936953,3,0.133422,14.99,NEWYEAR
3,2016-01-03,5435.051020,16,138936951,213,0.000000,0.69,NaN
4,2016-01-03,5435.051020,17,138936953,3,0.000000,14.99,NaN
5,2016-01-03,5435.051020,159,138936952,109,0.000000,3.99,NaN
6,2016-01-10,5335.275510,18,138936952,92,0.000000,3.99,NaN
7,2016-01-10,5335.275510,441,138936951,183,0.000000,0.69,NaN
8,2016-01-10,5335.275510,1904,138936953,6,0.000000,14.99,NaN
9,2016-01-17,5262.316327,247,138936952,86,0.000000,3.99,NaN


In [49]:
weekly_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316 entries, 0 to 315
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tran_wk             316 non-null    datetime64[ns]
 1   seas_index          316 non-null    float64       
 2   index               316 non-null    int64         
 3   prod_id             316 non-null    int64         
 4   weekly_sale_qty     316 non-null    int64         
 5   weekly_avg_disc     316 non-null    float64       
 6   weekly_shelf_price  316 non-null    float64       
 7   holiday             106 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 22.2+ KB


In [50]:
# Marketing Vehicles
# Now we want to include all marketing vehicles in our model input data
# Only flyer and store display were not applied to all three products. Thus, we merge them to the
# model input data separately from the other vehicles.
# Flyer

Flyer = promo[promo.vehicle == "Flyer"]
Flyer = Flyer[['tran_wk','amount','prod_assoc']]
Flyer.rename(columns={'amount':'Flyer', 'prod_assoc':'prod_id'}, inplace=True)
Flyer = Flyer.reset_index(drop=True)
Flyer['prod_id'] = Flyer['prod_id'].astype('int')
weekly_data = weekly_data.merge(Flyer, how='left', on=['tran_wk', 'prod_id'] )

In [51]:
# Store Display

Store_Display = promo[promo.vehicle == "Store Display"]
Store_Display = Store_Display[['tran_wk','amount','prod_assoc']]
Store_Display.rename(columns={'amount':'Store_Display', 'prod_assoc':'prod_id'}, inplace=True)
Store_Display = Store_Display.reset_index(drop=True)
Store_Display['prod_id'] = Store_Display['prod_id'].astype('int')
weekly_data = weekly_data.merge(Store_Display, how='left', on=['tran_wk', 'prod_id'] )

In [52]:
# Additional vehicles

In [53]:
# Paid Search
Paid_Search = promo[promo.vehicle == "Paid Search"]
Paid_Search = Paid_Search[['tran_wk','amount']]
Paid_Search.rename(columns={'amount':'Paid_Search'}, inplace=True)

In [54]:
# Web Dislay
Web_Display = promo[promo.vehicle == "Web Display"]
Web_Display = Web_Display[['tran_wk','amount']]
Web_Display.rename(columns={'amount':'Web_Display'}, inplace=True)

In [55]:
# Email
Email = promo[promo.vehicle == "Email"]
Email = Email[['tran_wk','amount']]
Email.rename(columns={'amount':'Email'}, inplace=True)

In [56]:
# TV & Radio

TV_Radio = TV_Radio[['tran_wk', 'TV', 'Radio']]

In [57]:
# Additional 

additional = TV_Radio.merge(Paid_Search, how='left', on=['tran_wk'])
additional = additional.merge(Web_Display, how='left', on=['tran_wk'])
additional = additional.merge(Email, how='left', on=['tran_wk'])

In [58]:
# Model input

model_input = weekly_data.merge(additional, how='left', on=['tran_wk']) 
model_input = model_input[model_input.weekly_sale_qty>=0]
model_input.fillna(0, inplace=True)

In [59]:
#We have data from 2015 that we need to remove - there is only one day worth of data from 2015 and that an skew our y variable (quantity). Also, there are limited promotions in this week
last_week_2015 = min(model_input.tran_wk)

In [60]:
model_input = model_input[model_input.tran_wk!= last_week_2015]

In [61]:
model_input.reset_index(drop=True, inplace=True)
model_input.drop('index', axis=1, inplace=True)

In [62]:
#On running the models we found there are multicollinearity issues with the data - XMAS, PrXMAS and New Year are multicollinear with e-mail
#After comparing sales in final weeks of 2016 and 2017 against the previous weeks, we found that PrXMAS and NEWYEAR see a surge in sales, but XMAS does not. Hence, we remove XMAS from the model

#Remove XMAS and then remodel for the 3 products

model_input.holiday = model_input.holiday.replace("XMAS", 0)

In [63]:
model_input

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0


In [64]:
model_input.dtypes

tran_wk               datetime64[ns]
seas_index                   float64
prod_id                        int64
weekly_sale_qty                int64
weekly_avg_disc              float64
weekly_shelf_price           float64
holiday                       object
Flyer                        float64
Store_Display                float64
TV                           float64
Radio                        float64
Paid_Search                    int64
Web_Display                  float64
Email                        float64
dtype: object

In [65]:
#Now run a model for each individual product predict weekly sales quantity, and then decompose it into the DueTos

# Duetos 138936951
model_138936951 = model_input[model_input['prod_id'] == 138936951]

model_138936951.shape

(106, 14)

In [66]:
# transform y

model_138936951['max_sales_qty'] = max(model_138936951.weekly_sale_qty) * 1.2
model_138936951['sales_trfm'] = np.log(model_138936951.weekly_sale_qty / (model_138936951.max_sales_qty - model_138936951.weekly_sale_qty))
model_138936951.fillna(0, inplace=True)

In [67]:
model_138936951

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,403.2,0.113216
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,403.2,-0.185050
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,403.2,-0.155072
10,2016-01-24,5252.102041,138936951,223,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,403.2,0.213104
12,2016-01-31,5327.520408,138936951,202,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,403.2,0.003968
15,2016-02-07,5516.459184,138936951,226,0.144928,0.69,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,403.2,0.243256
19,2016-02-14,5686.591837,138936951,170,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,403.2,-0.316098
21,2016-02-21,5867.561224,138936951,240,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,403.2,0.385662
24,2016-02-28,6017.255102,138936951,148,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,403.2,-0.544835
27,2016-03-06,6209.806122,138936951,208,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,403.2,0.063513


In [68]:
#building regressions. Using sales qty as y and all other promotion ads as Xs

from statsmodels.formula.api import ols

formula = "sales_trfm ~ weekly_avg_disc + weekly_shelf_price + Flyer +  Web_Display + Paid_Search + Email+ TV + Radio + seas_index + holiday"  # no Store Display for 138936951

lm = ols(formula, data = model_138936951)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             sales_trfm   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     4.071
Date:                Fri, 16 Apr 2021   Prob (F-statistic):           7.49e-07
Time:                        08:51:38   Log-Likelihood:                -36.723
No. Observations:                 106   AIC:                             127.4
Df Residuals:                      79   BIC:                             199.4
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   1.5276      0.954      1.601      0.113      -0.371       3.426
holiday[T.ALLSAINTS]       -0.2401      0.297     -0.808      0.421      -0.832       0.351
holiday[T.ASSUMPTION]       0.1131      0.298      0.380      0.705      -0.480       0.706
holiday[T.CARNIVAL]        -0.6401      0.457     -1.400      0.166      -1.550       0.270
holiday[T.CORPUS]          -0.4895      0.306     -1.600      0.114      -1.098       0.119
holiday[T.EASTER]          -0.3343      0.305     -1.095      0.277      -0.942       0.273
holiday[T.IMMACULATE]       0.0759      0.297      0.255      0.799      -0.516       0.668
holiday[T.LABOR]           -1.0099      0.373     -2.707      0.008      -1.752      -0.267
holiday[T.LIBERTY]         -0.8646      0.373     -2.317      0.023      -1.607      -0.122
holiday[T.NEWYEAR]         -1.9260      0.397     -4.851      0.000      -2.716      -1.136
holiday[T.POPEVISIT]       -0.3339      0.658     -0.508      0.613      -1.643       0.975
holiday[T.PORTUGAL]        -0.8144      0.595     -1.369      0.175      -1.999       0.370
holiday[T.PrASSUMPTION]    -0.2385      0.316     -0.754      0.453      -0.868       0.391
holiday[T.PrEASTER]        -0.6867      0.336     -2.045      0.044      -1.355      -0.018
holiday[T.PrLIBERTY]       -0.6187      0.320     -1.933      0.057      -1.256       0.018
holiday[T.PrXMAS]           0.2356      0.410      0.574      0.567      -0.581       1.052
holiday[T.REPUBLIC]        -1.5769      0.522     -3.021      0.003      -2.616      -0.538
holiday[T.RESTORATION]     -0.4338      0.360     -1.207      0.231      -1.149       0.282
weekly_avg_disc             6.2865      1.650      3.810      0.000       3.002       9.571
weekly_shelf_price         -2.6823      1.301     -2.061      0.043      -5.273      -0.092
Flyer                       0.0198      0.427      0.046      0.963      -0.831       0.870
Web_Display             -5.522e-07      5e-07     -1.104      0.273   -1.55e-06    4.44e-07
Paid_Search             -8.878e-07   3.19e-06     -0.278      0.781   -7.23e-06    5.46e-06
Email                    2.339e-07   1.82e-06      0.128      0.898   -3.39e-06    3.86e-06
TV                          0.6479      0.810      0.800      0.426      -0.965       2.260
Radio                      -0.2529      0.605     -0.418      0.677      -1.458       0.952
seas_index               5.628e-05   3.19e-05      1.764      0.082   -7.21e-06       0.000
==============================================================================
Omnibus:                       24.425   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              206.378
Skew:                          -0.007   Prob(JB):                     1.53e-45
Kurtosi

In [69]:
#use logit function to bound the prediction, and present the interaction more clearly, since audience might exosed many different media such as radio and tv

model_138936951['pred'] = results.predict()

model_138936951['pred'] = model_138936951.max_sales_qty * np.exp(model_138936951.pred) / (np.exp(model_138936951.pred) + 1)

In [70]:
from sklearn.metrics import mean_squared_error as mse

In [71]:
mse(model_138936951['weekly_sale_qty'], model_138936951['pred'], squared=False)

30.05021730608029

In [72]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant

In [141]:
X = model_138936951[["weekly_avg_disc","weekly_shelf_price","Flyer","Web_Display","Paid_Search","Email","TV","Radio","seas_index","holiday"]].copy()
X = pd.get_dummies(X, columns=["holiday"], sparse=False, drop_first=True)
X = add_constant(X)
pd.Series([vif(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

const                   245.622288
weekly_avg_disc           3.919002
weekly_shelf_price        0.776586
Flyer                     2.214345
Web_Display               0.539368
Paid_Search               0.553606
Email                     1.911634
TV                        4.358654
Radio                     3.318540
seas_index                1.224964
holiday_ALLSAINTS         0.441143
holiday_ASSUMPTION        0.443462
holiday_CARNIVAL          1.045031
holiday_CORPUS            0.467617
holiday_EASTER            0.465812
holiday_IMMACULATE        0.442145
holiday_LABOR             0.695306
holiday_LIBERTY           0.695893
holiday_NEWYEAR           0.787561
holiday_POPEVISIT         1.091527
holiday_PORTUGAL          1.768888
holiday_PrASSUMPTION      0.499479
holiday_PrEASTER          0.563235
holiday_PrLIBERTY         0.511849
holiday_PrXMAS            0.840174
holiday_REPUBLIC          1.361415
holiday_RESTORATION       0.645816
dtype: float64

In [74]:
model_138936951

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,403.2,0.113216,195.592733
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,403.2,-0.185050,196.671764
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,403.2,-0.155072,196.077043
10,2016-01-24,5252.102041,138936951,223,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,403.2,0.213104,196.619850
12,2016-01-31,5327.520408,138936951,202,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,403.2,0.003968,198.958577
15,2016-02-07,5516.459184,138936951,226,0.144928,0.69,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,403.2,0.243256,225.956051
19,2016-02-14,5686.591837,138936951,170,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,403.2,-0.316098,199.184144
21,2016-02-21,5867.561224,138936951,240,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,403.2,0.385662,199.044990
24,2016-02-28,6017.255102,138936951,148,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,403.2,-0.544835,203.016978
27,2016-03-06,6209.806122,138936951,208,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,403.2,0.063513,202.238863


In [75]:
# decomposit DueTo
# baseline is base price + seasonailty + holiday, set the price first appeared in the time period as the base price

model_pred = model_138936951.copy()

In [76]:
model_pred[['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']] = 0

In [77]:
model_138936951['base'] = results.predict(model_pred)

In [78]:
model_138936951['base'] = model_138936951.max_sales_qty * np.exp(model_138936951.base) / (np.exp(model_138936951.base) + 1)

In [79]:
model_138936951

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,403.2,0.113216,195.592733,199.862744
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,403.2,-0.185050,196.671764,199.296772
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,403.2,-0.155072,196.077043,198.882937
10,2016-01-24,5252.102041,138936951,223,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,403.2,0.213104,196.619850,198.825002
12,2016-01-31,5327.520408,138936951,202,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,403.2,0.003968,198.958577,199.252783
15,2016-02-07,5516.459184,138936951,226,0.144928,0.69,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,403.2,0.243256,225.956051,138.045629
19,2016-02-14,5686.591837,138936951,170,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,403.2,-0.316098,199.184144,201.289699
21,2016-02-21,5867.561224,138936951,240,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,403.2,0.385662,199.044990,202.316339
24,2016-02-28,6017.255102,138936951,148,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,403.2,-0.544835,203.016978,203.165528
27,2016-03-06,6209.806122,138936951,208,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,403.2,0.063513,202.238863,204.257752


In [80]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    model_pred = model_138936951.copy()
    model_pred[i] = 0
    tmp = i + "_base"
    model_138936951[tmp] = results.predict(model_pred)
    model_138936951[tmp] = model_138936951.max_sales_qty * np.exp(model_138936951[tmp]) / (np.exp(model_138936951[tmp]) + 1)
    model_138936951[tmp] = model_138936951["pred"] - model_138936951[tmp]

In [81]:
model_138936951

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,403.2,0.113216,195.592733,199.862744,0.000000,0.000000,0.000000,-4.270011,0.000000,0.000000,0.000000
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,403.2,-0.185050,196.671764,199.296772,0.000000,0.000000,0.000000,-2.625008,0.000000,0.000000,0.000000
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,403.2,-0.155072,196.077043,198.882937,0.000000,0.000000,0.000000,-2.805894,0.000000,0.000000,0.000000
10,2016-01-24,5252.102041,138936951,223,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,403.2,0.213104,196.619850,198.825002,0.000000,0.000000,0.000000,-2.205153,0.000000,0.000000,0.000000
12,2016-01-31,5327.520408,138936951,202,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,403.2,0.003968,198.958577,199.252783,0.000000,0.000000,0.000000,-0.294206,0.000000,0.000000,0.000000
15,2016-02-07,5516.459184,138936951,226,0.144928,0.69,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,403.2,0.243256,225.956051,138.045629,89.317984,0.000000,0.000000,-1.542398,0.000000,0.000000,0.000000
19,2016-02-14,5686.591837,138936951,170,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,403.2,-0.316098,199.184144,201.289699,0.000000,0.000000,0.000000,-2.105554,0.000000,0.000000,0.000000
21,2016-02-21,5867.561224,138936951,240,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,403.2,0.385662,199.044990,202.316339,0.000000,0.000000,0.000000,-3.271349,0.000000,0.000000,0.000000
24,2016-02-28,6017.255102,138936951,148,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,403.2,-0.544835,203.016978,203.165528,0.000000,0.000000,0.000000,-0.148549,0.000000,0.000000,0.000000
27,2016-03-06,6209.806122,138936951,208,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,403.2,0.063513,202.238863,204.257752,0.000000,0.000000,0.000000,-2.018889,0.000000,0.000000,0.000000


In [82]:
model_138936951['sum'] = model_138936951['base'] + model_138936951['weekly_avg_disc_base'] + model_138936951['Email_base'] + model_138936951['Flyer_base'] + model_138936951['Paid_Search_base'] + model_138936951['Web_Display_base'] + model_138936951['TV_base'] + model_138936951['Radio_base']

In [83]:
model_138936951['base'] = (model_138936951['base']*model_138936951['weekly_sale_qty'])/model_138936951['sum']

In [84]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    tmp = i + "_base"
    model_138936951[tmp] = (model_138936951[tmp]*model_138936951['weekly_sale_qty'])/model_138936951['sum']

In [85]:
model_138936951

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base,sum
0,2016-01-03,5435.051020,138936951,213,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,403.2,0.113216,195.592733,217.650031,0.000000,0.000000,0.000000,-4.650031,0.000000,0.000000,0.000000,195.592733
4,2016-01-10,5335.275510,138936951,183,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,403.2,-0.185050,196.671764,185.442529,0.000000,0.000000,0.000000,-2.442529,0.000000,0.000000,0.000000,196.671764
7,2016-01-17,5262.316327,138936951,186,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,403.2,-0.155072,196.077043,188.661690,0.000000,0.000000,0.000000,-2.661690,0.000000,0.000000,0.000000,196.077043
10,2016-01-24,5252.102041,138936951,223,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,403.2,0.213104,196.619850,225.501014,0.000000,0.000000,0.000000,-2.501014,0.000000,0.000000,0.000000,196.619850
12,2016-01-31,5327.520408,138936951,202,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,403.2,0.003968,198.958577,202.298704,0.000000,0.000000,0.000000,-0.298704,0.000000,0.000000,0.000000,198.958577
15,2016-02-07,5516.459184,138936951,226,0.144928,0.69,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,403.2,0.243256,225.956051,138.154921,89.388698,0.000000,0.000000,-1.543619,0.000000,0.000000,0.000000,225.821215
19,2016-02-14,5686.591837,138936951,170,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,403.2,-0.316098,199.184144,171.797052,0.000000,0.000000,0.000000,-1.797052,0.000000,0.000000,0.000000,199.184144
21,2016-02-21,5867.561224,138936951,240,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,403.2,0.385662,199.044990,243.944454,0.000000,0.000000,0.000000,-3.944454,0.000000,0.000000,0.000000,199.044990
24,2016-02-28,6017.255102,138936951,148,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,403.2,-0.544835,203.016978,148.108293,0.000000,0.000000,0.000000,-0.108293,0.000000,0.000000,0.000000,203.016978
27,2016-03-06,6209.806122,138936951,208,0.000000,0.69,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,403.2,0.063513,202.238863,210.076401,0.000000,0.000000,0.000000,-2.076401,0.000000,0.000000,0.000000,202.238863


In [86]:
# repeat the process for 138936952

model_138936952 = model_input[model_input['prod_id'] == 138936952]

model_138936952.shape

(106, 14)

In [87]:
# transform y

model_138936952['max_sales_qty'] = max(model_138936952.weekly_sale_qty) * 1.2
model_138936952['sales_trfm'] = np.log(model_138936952.weekly_sale_qty / (model_138936952.max_sales_qty - model_138936952.weekly_sale_qty))
model_138936952.fillna(0, inplace=True)

In [88]:
model_138936952

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,259.2,-0.320620
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,259.2,-0.597402
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,259.2,-0.700100
11,2016-01-24,5252.102041,138936952,91,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,259.2,-0.614294
13,2016-01-31,5327.520408,138936952,108,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,259.2,-0.336472
16,2016-02-07,5516.459184,138936952,131,0.125313,3.99,CARNIVAL,0.0,1.0,0.000000,0.000000,17507,0.0,0.0,259.2,0.021606
18,2016-02-14,5686.591837,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,259.2,-0.879845
22,2016-02-21,5867.561224,138936952,94,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,259.2,-0.563862
25,2016-02-28,6017.255102,138936952,85,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,259.2,-0.717553
28,2016-03-06,6209.806122,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,259.2,-0.879845


In [89]:
#building regressions. Using sales qty as y and all other promotion ads as Xs

from statsmodels.formula.api import ols

formula = "sales_trfm ~ weekly_avg_disc + weekly_shelf_price + Flyer +  Web_Display + Paid_Search + Email+ TV + Radio + seas_index + holiday"  # no Store Display for 138936951

lm = ols(formula, data = model_138936952)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             sales_trfm   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     9.098
Date:                Fri, 16 Apr 2021   Prob (F-statistic):           1.11e-14
Time:                        08:51:39   Log-Likelihood:                -10.767
No. Observations:                 106   AIC:                             75.53
Df Residuals:                      79   BIC:                             147.4
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   1.5034      0.723      2.080      0.041       0.065       2.942
holiday[T.ALLSAINTS]       -0.3360      0.237     -1.416      0.161      -0.808       0.136
holiday[T.ASSUMPTION]       0.2153      0.234      0.922      0.359      -0.249       0.680
holiday[T.CARNIVAL]        -0.1484      0.315     -0.472      0.638      -0.775       0.478
holiday[T.CORPUS]           0.0780      0.244      0.320      0.750      -0.407       0.563
holiday[T.EASTER]          -0.3145      0.235     -1.340      0.184      -0.782       0.153
holiday[T.IMMACULATE]       0.2084      0.233      0.894      0.374      -0.255       0.672
holiday[T.LABOR]            0.1533      0.270      0.568      0.571      -0.384       0.690
holiday[T.LIBERTY]         -0.3663      0.270     -1.354      0.179      -0.905       0.172
holiday[T.NEWYEAR]         -0.1469      0.318     -0.462      0.646      -0.780       0.486
holiday[T.POPEVISIT]       -0.1482      0.395     -0.375      0.709      -0.935       0.639
holiday[T.PORTUGAL]         0.5953      0.358      1.662      0.100      -0.117       1.308
holiday[T.PrASSUMPTION]    -0.1562      0.253     -0.617      0.539      -0.660       0.348
holiday[T.PrEASTER]        -0.1125      0.267     -0.421      0.675      -0.644       0.419
holiday[T.PrLIBERTY]       -0.3508      0.244     -1.440      0.154      -0.836       0.134
holiday[T.PrXMAS]           0.0572      0.331      0.173      0.863      -0.601       0.715
holiday[T.REPUBLIC]         0.2391      0.352      0.678      0.500      -0.462       0.941
holiday[T.RESTORATION]      0.3829      0.261      1.468      0.146      -0.136       0.902
weekly_avg_disc             4.1033      1.371      2.992      0.004       1.374       6.833
weekly_shelf_price         -0.7646      0.173     -4.419      0.000      -1.109      -0.420
Flyer                       0.0146      0.143      0.102      0.919      -0.270       0.299
Web_Display             -6.812e-07   3.95e-07     -1.725      0.088   -1.47e-06    1.05e-07
Paid_Search              1.396e-06    2.5e-06      0.558      0.579   -3.59e-06    6.38e-06
Email                    -2.81e-07   1.32e-06     -0.212      0.832   -2.92e-06    2.35e-06
TV                         -0.8800      0.630     -1.398      0.166      -2.133       0.373
Radio                       0.3798      0.464      0.819      0.415      -0.543       1.303
seas_index                  0.0002   2.55e-05      6.400      0.000       0.000       0.000
==============================================================================
Omnibus:                        0.387   Durbin-Watson:                   2.181
Prob(Omnibus):                  0.824   Jarque-Bera (JB):                0.550
Skew:                          -0.060   Prob(JB):                        0.759
Kurtosi

In [90]:
#use logit function to bound the prediction, and present the interaction more clearly, since audience might exosed many different media such as radio and tv

model_138936952['pred'] = results.predict()

model_138936952['pred'] = model_138936952.max_sales_qty * np.exp(model_138936952.pred) / (np.exp(model_138936952.pred) + 1)

In [91]:
from sklearn.metrics import mean_squared_error as mse

In [92]:
mse(model_138936952['weekly_sale_qty'], model_138936952['pred'], squared=False)

15.658406682796674

In [93]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant

In [140]:
X = model_138936952[["weekly_avg_disc","weekly_shelf_price","Flyer","Web_Display","Paid_Search","Email","TV","Radio","seas_index","holiday"]].copy()
X = pd.get_dummies(X, columns=["holiday"], sparse=False, drop_first=True)
X = add_constant(X)
pd.Series([vif(X.values, i)
               for i in range(X.shape[1])], 
              index=X.columns)

const                   230.196959
weekly_avg_disc           3.674592
weekly_shelf_price        0.734346
Flyer                     1.480571
Web_Display               0.548252
Paid_Search               0.557090
Email                     1.649511
TV                        4.295409
Radio                     3.178684
seas_index                1.275361
holiday_ALLSAINTS         0.458841
holiday_ASSUMPTION        0.444647
holiday_CARNIVAL          0.807484
holiday_CORPUS            0.484732
holiday_EASTER            0.449112
holiday_IMMACULATE        0.442480
holiday_LABOR             0.593606
holiday_LIBERTY           0.596545
holiday_NEWYEAR           0.824983
holiday_POPEVISIT         0.643664
holiday_PORTUGAL          1.045671
holiday_PrASSUMPTION      0.522963
holiday_PrEASTER          0.580586
holiday_PrLIBERTY         0.483834
holiday_PrXMAS            0.891400
holiday_REPUBLIC          1.013051
holiday_RESTORATION       0.555203
dtype: float64

In [95]:
model_138936952

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,259.2,-0.320620,92.166839
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,259.2,-0.597402,89.690906
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,259.2,-0.700100,89.159712
11,2016-01-24,5252.102041,138936952,91,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,259.2,-0.614294,88.514894
13,2016-01-31,5327.520408,138936952,108,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,259.2,-0.336472,87.496111
16,2016-02-07,5516.459184,138936952,131,0.125313,3.99,CARNIVAL,0.0,1.0,0.000000,0.000000,17507,0.0,0.0,259.2,0.021606,112.989194
18,2016-02-14,5686.591837,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,259.2,-0.879845,92.596210
22,2016-02-21,5867.561224,138936952,94,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,259.2,-0.563862,95.453808
25,2016-02-28,6017.255102,138936952,85,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,259.2,-0.717553,93.992488
28,2016-03-06,6209.806122,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,259.2,-0.879845,97.650874


In [96]:
# decomposit DueTo
# baseline is base price + seasonailty + holiday, set the price first appeared in the time period as the base price

model_pred = model_138936952.copy()

In [97]:
model_pred[['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']] = 0

In [98]:
model_138936952['base'] = results.predict(model_pred)

In [99]:
model_138936952['base'] = model_138936952.max_sales_qty * np.exp(model_138936952.base) / (np.exp(model_138936952.base) + 1)

In [100]:
model_138936952

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,259.2,-0.320620,92.166839,88.247839
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,259.2,-0.597402,89.690906,87.303434
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,259.2,-0.700100,89.159712,86.616015
11,2016-01-24,5252.102041,138936952,91,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,259.2,-0.614294,88.514894,86.519991
13,2016-01-31,5327.520408,138936952,108,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,259.2,-0.336472,87.496111,87.230238
16,2016-02-07,5516.459184,138936952,131,0.125313,3.99,CARNIVAL,0.0,1.0,0.000000,0.000000,17507,0.0,0.0,259.2,0.021606,112.989194,80.558317
18,2016-02-14,5686.591837,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,259.2,-0.879845,92.596210,90.650363
22,2016-02-21,5867.561224,138936952,94,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,259.2,-0.563862,95.453808,92.397357
25,2016-02-28,6017.255102,138936952,85,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,259.2,-0.717553,93.992488,93.853687
28,2016-03-06,6209.806122,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,259.2,-0.879845,97.650874,95.741379


In [101]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    model_pred = model_138936952.copy()
    model_pred[i] = 0
    tmp = i + "_base"
    model_138936952[tmp] = results.predict(model_pred)
    model_138936952[tmp] = model_138936952.max_sales_qty * np.exp(model_138936952[tmp]) / (np.exp(model_138936952[tmp]) + 1)
    model_138936952[tmp] = model_138936952["pred"] - model_138936952[tmp]

In [102]:
model_138936952

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,259.2,-0.320620,92.166839,88.247839,0.000000,0.000000,0.000000,3.919000,0.000000,0.000000,0.000000
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,259.2,-0.597402,89.690906,87.303434,0.000000,0.000000,0.000000,2.387471,0.000000,0.000000,0.000000
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,259.2,-0.700100,89.159712,86.616015,0.000000,0.000000,0.000000,2.543697,0.000000,0.000000,0.000000
11,2016-01-24,5252.102041,138936952,91,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,259.2,-0.614294,88.514894,86.519991,0.000000,0.000000,0.000000,1.994903,0.000000,0.000000,0.000000
13,2016-01-31,5327.520408,138936952,108,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,259.2,-0.336472,87.496111,87.230238,0.000000,0.000000,0.000000,0.265873,0.000000,0.000000,0.000000
16,2016-02-07,5516.459184,138936952,131,0.125313,3.99,CARNIVAL,0.0,1.0,0.000000,0.000000,17507,0.0,0.0,259.2,0.021606,112.989194,80.558317,31.067568,0.000000,0.000000,1.555338,0.000000,0.000000,0.000000
18,2016-02-14,5686.591837,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,259.2,-0.879845,92.596210,90.650363,0.000000,0.000000,0.000000,1.945848,0.000000,0.000000,0.000000
22,2016-02-21,5867.561224,138936952,94,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,259.2,-0.563862,95.453808,92.397357,0.000000,0.000000,0.000000,3.056450,0.000000,0.000000,0.000000
25,2016-02-28,6017.255102,138936952,85,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,259.2,-0.717553,93.992488,93.853687,0.000000,0.000000,0.000000,0.138800,0.000000,0.000000,0.000000
28,2016-03-06,6209.806122,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,259.2,-0.879845,97.650874,95.741379,0.000000,0.000000,0.000000,1.909495,0.000000,0.000000,0.000000


In [103]:
model_138936952['sum'] = model_138936952['base'] + model_138936952['weekly_avg_disc_base'] + model_138936952['Email_base'] + model_138936952['Flyer_base'] + model_138936952['Paid_Search_base'] + model_138936952['Web_Display_base'] + model_138936952['TV_base'] + model_138936952['Radio_base']

In [104]:
model_138936952['base'] = (model_138936952['base']*model_138936952['weekly_sale_qty'])/model_138936952['sum']

In [105]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    tmp = i + "_base"
    model_138936952[tmp] = (model_138936952[tmp]*model_138936952['weekly_sale_qty'])/model_138936952['sum']

In [106]:
model_138936952

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base,sum
2,2016-01-03,5435.051020,138936952,109,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,259.2,-0.320620,92.166839,104.365242,0.000000,0.000000,0.000000,4.634758,0.000000,0.000000,0.000000,92.166839
3,2016-01-10,5335.275510,138936952,92,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,259.2,-0.597402,89.690906,89.551063,0.000000,0.000000,0.000000,2.448937,0.000000,0.000000,0.000000,89.690906
6,2016-01-17,5262.316327,138936952,86,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,259.2,-0.700100,89.159712,83.546449,0.000000,0.000000,0.000000,2.453551,0.000000,0.000000,0.000000,89.159712
11,2016-01-24,5252.102041,138936952,91,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,259.2,-0.614294,88.514894,88.949089,0.000000,0.000000,0.000000,2.050911,0.000000,0.000000,0.000000,88.514894
13,2016-01-31,5327.520408,138936952,108,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,259.2,-0.336472,87.496111,107.671822,0.000000,0.000000,0.000000,0.328178,0.000000,0.000000,0.000000,87.496111
16,2016-02-07,5516.459184,138936952,131,0.125313,3.99,CARNIVAL,0.0,1.0,0.000000,0.000000,17507,0.0,0.0,259.2,0.021606,112.989194,93.241080,35.958716,0.000000,0.000000,1.800204,0.000000,0.000000,0.000000,113.181223
18,2016-02-14,5686.591837,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,259.2,-0.879845,92.596210,74.402911,0.000000,0.000000,0.000000,1.597089,0.000000,0.000000,0.000000,92.596210
22,2016-02-21,5867.561224,138936952,94,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,259.2,-0.563862,95.453808,90.990101,0.000000,0.000000,0.000000,3.009899,0.000000,0.000000,0.000000,95.453808
25,2016-02-28,6017.255102,138936952,85,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,259.2,-0.717553,93.992488,84.874479,0.000000,0.000000,0.000000,0.125521,0.000000,0.000000,0.000000,93.992488
28,2016-03-06,6209.806122,138936952,76,0.000000,3.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,259.2,-0.879845,97.650874,74.513873,0.000000,0.000000,0.000000,1.486127,0.000000,0.000000,0.000000,97.650874


In [107]:
# repeat the process for 138936953

model_138936953 = model_input[model_input['prod_id'] == 138936953]

model_138936953.shape

(101, 14)

In [108]:
# transform y

model_138936953['max_sales_qty'] = max(model_138936953.weekly_sale_qty) * 1.2
model_138936953['sales_trfm'] = np.log(model_138936953.weekly_sale_qty / (model_138936953.max_sales_qty - model_138936953.weekly_sale_qty))
model_138936953.fillna(0, inplace=True)

In [109]:
model_138936953

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,97.2,-3.446808
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,97.2,-2.721295
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,97.2,-2.721295
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,97.2,-3.446808
14,2016-01-31,5327.520408,138936953,8,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,97.2,-2.411439
17,2016-02-07,5516.459184,138936953,12,0.133422,14.99,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,97.2,-1.960095
20,2016-02-14,5686.591837,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,97.2,-3.446808
23,2016-02-21,5867.561224,138936953,7,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,97.2,-2.556119
26,2016-02-28,6017.255102,138936953,2,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,97.2,-3.862833
29,2016-03-06,6209.806122,138936953,4,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,97.2,-3.148453


In [110]:
#building regressions. Using sales qty as y and all other promotion ads as Xs

from statsmodels.formula.api import ols

formula = "sales_trfm ~ weekly_avg_disc + weekly_shelf_price + Flyer +  Web_Display + Paid_Search + Email+ TV + Radio + seas_index + holiday"  # no Store Display for 138936951

lm = ols(formula, data = model_138936953)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             sales_trfm   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     13.24
Date:                Fri, 16 Apr 2021   Prob (F-statistic):           1.41e-18
Time:                        08:51:40   Log-Likelihood:                -70.853
No. Observations:                 101   AIC:                             195.7
Df Residuals:                      74   BIC:                             266.3
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.6477      1.549      4.936      0.000       4.561      10.735
holiday[T.ALLSAINTS]       -0.2166      0.440     -0.492      0.624      -1.093       0.660
holiday[T.ASSUMPTION]      -0.2542      0.429     -0.593      0.555      -1.109       0.601
holiday[T.CARNIVAL]        -1.0242      0.806     -1.271      0.208      -2.630       0.582
holiday[T.CORPUS]          -0.2918      0.449     -0.649      0.518      -1.187       0.604
holiday[T.EASTER]           0.3916      0.434      0.902      0.370      -0.473       1.256
holiday[T.IMMACULATE]       0.0740      0.587      0.126      0.900      -1.095       1.243
holiday[T.LABOR]            1.3004      0.726      1.791      0.077      -0.147       2.747
holiday[T.LIBERTY]         -0.7880      0.678     -1.162      0.249      -2.140       0.564
holiday[T.NEWYEAR]          0.1096      0.928      0.118      0.906      -1.739       1.959
holiday[T.POPEVISIT]       -0.4089      0.945     -0.433      0.666      -2.292       1.474
holiday[T.PORTUGAL]        -1.1877      0.907     -1.309      0.194      -2.995       0.620
holiday[T.PrASSUMPTION]     0.7583      0.482      1.573      0.120      -0.202       1.719
holiday[T.PrEASTER]        -0.0375      0.577     -0.065      0.948      -1.187       1.112
holiday[T.PrLIBERTY]        0.6364      0.480      1.327      0.189      -0.319       1.592
holiday[T.PrXMAS]           0.0841      0.825      0.102      0.919      -1.560       1.728
holiday[T.REPUBLIC]        -1.3744      0.897     -1.532      0.130      -3.162       0.413
holiday[T.RESTORATION]      0.0991      0.603      0.165      0.870      -1.101       1.300
weekly_avg_disc            12.4609      4.225      2.950      0.004       4.043      20.879
weekly_shelf_price         -0.8341      0.101     -8.292      0.000      -1.035      -0.634
Flyer                      -0.8640      0.735     -1.175      0.244      -2.329       0.601
Web_Display              4.249e-07   7.51e-07      0.566      0.573   -1.07e-06    1.92e-06
Paid_Search              4.795e-06   4.85e-06      0.989      0.326   -4.86e-06    1.45e-05
Email                   -3.076e-07   3.33e-06     -0.092      0.927   -6.95e-06    6.33e-06
TV                          0.6950      1.183      0.588      0.559      -1.662       3.052
Radio                       0.3596      0.877      0.410      0.683      -1.388       2.107
seas_index                  0.0003   4.79e-05      6.751      0.000       0.000       0.000
==============================================================================
Omnibus:                        1.939   Durbin-Watson:                   2.309
Prob(Omnibus):                  0.379   Jarque-Bera (JB):                1.548
Skew:                          -0.298   Prob(JB):                        0.461
Kurtosi

In [111]:
#use logit function to bound the prediction, and present the interaction more clearly, since audience might exosed many different media such as radio and tv

model_138936953['pred'] = results.predict()

model_138936953['pred'] = model_138936953.max_sales_qty * np.exp(model_138936953.pred) / (np.exp(model_138936953.pred) + 1)

In [112]:
mse(model_138936953['weekly_sale_qty'], model_138936953['pred'], squared=False)

3.9562696779967736

In [142]:
X = model_138936953[["weekly_avg_disc","weekly_shelf_price","Flyer","Web_Display","Paid_Search","Email","TV","Radio","seas_index","holiday"]].copy()
X = pd.get_dummies(X, columns=["holiday"], sparse=False, drop_first=True)
X = add_constant(X)
pd.Series([vif(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

const                   298.317772
weekly_avg_disc          14.396976
weekly_shelf_price        0.762764
Flyer                     8.024113
Web_Display               0.582050
Paid_Search               0.574833
Email                     3.085740
TV                        4.452333
Radio                     3.318196
seas_index                1.292664
holiday_ALLSAINTS         0.467035
holiday_ASSUMPTION        0.443877
holiday_CARNIVAL          1.566745
holiday_CORPUS            0.487276
holiday_EASTER            0.454452
holiday_IMMACULATE        0.419383
holiday_LABOR             0.642530
holiday_LIBERTY           1.110213
holiday_NEWYEAR           2.077274
holiday_POPEVISIT         1.088021
holiday_PORTUGAL          1.985264
holiday_PrASSUMPTION      0.560571
holiday_PrEASTER          0.803406
holiday_PrLIBERTY         0.554692
holiday_PrXMAS            1.641836
holiday_REPUBLIC          1.941814
holiday_RESTORATION       0.875795
dtype: float64

In [114]:
model_138936953

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,97.2,-3.446808,5.208167
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,97.2,-2.721295,4.645517
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,97.2,-2.721295,4.584599
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,97.2,-3.446808,4.431949
14,2016-01-31,5327.520408,138936953,8,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,97.2,-2.411439,4.113170
17,2016-02-07,5516.459184,138936953,12,0.133422,14.99,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,97.2,-1.960095,8.448868
20,2016-02-14,5686.591837,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,97.2,-3.446808,5.039228
23,2016-02-21,5867.561224,138936953,7,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,97.2,-2.556119,5.649348
26,2016-02-28,6017.255102,138936953,2,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,97.2,-3.862833,5.048539
29,2016-03-06,6209.806122,138936953,4,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,97.2,-3.148453,5.885017


In [115]:
# decomposit DueTo
# baseline is base price + seasonailty + holiday, set the price first appeared in the time period as the base price

model_pred = model_138936953.copy()

In [116]:
model_pred[['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']] = 0

In [117]:
model_138936953['base'] = results.predict(model_pred)

In [118]:
model_138936953['base'] = model_138936953.max_sales_qty * np.exp(model_138936953.base) / (np.exp(model_138936953.base) + 1)

In [119]:
model_138936953

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,97.2,-3.446808,5.208167,4.188545
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,97.2,-2.721295,4.645517,4.061246
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,97.2,-2.721295,4.584599,3.970511
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,97.2,-3.446808,4.431949,3.957964
14,2016-01-31,5327.520408,138936953,8,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,97.2,-2.411439,4.113170,4.051508
17,2016-02-07,5516.459184,138936953,12,0.133422,14.99,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,97.2,-1.960095,8.448868,1.587236
20,2016-02-14,5686.591837,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,97.2,-3.446808,5.039228,4.526593
23,2016-02-21,5867.561224,138936953,7,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,97.2,-2.556119,5.649348,4.785689
26,2016-02-28,6017.255102,138936953,2,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,97.2,-3.862833,5.048539,5.010574
29,2016-03-06,6209.806122,138936953,4,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,97.2,-3.148453,5.885017,5.314554


In [120]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    model_pred = model_138936953.copy()
    model_pred[i] = 0
    tmp = i + "_base"
    model_138936953[tmp] = results.predict(model_pred)
    model_138936953[tmp] = model_138936953.max_sales_qty * np.exp(model_138936953[tmp]) / (np.exp(model_138936953[tmp]) + 1)
    model_138936953[tmp] = model_138936953["pred"] - model_138936953[tmp]

In [121]:
model_138936953

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,97.2,-3.446808,5.208167,4.188545,0.000000,0.000000,0.000000,1.019622,0.000000,0.000000,0.000000
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,97.2,-2.721295,4.645517,4.061246,0.000000,0.000000,0.000000,0.584271,0.000000,0.000000,0.000000
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,97.2,-2.721295,4.584599,3.970511,0.000000,0.000000,0.000000,0.614088,0.000000,0.000000,0.000000
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,97.2,-3.446808,4.431949,3.957964,0.000000,0.000000,0.000000,0.473984,0.000000,0.000000,0.000000
14,2016-01-31,5327.520408,138936953,8,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,97.2,-2.411439,4.113170,4.051508,0.000000,0.000000,0.000000,0.061662,0.000000,0.000000,0.000000
17,2016-02-07,5516.459184,138936953,12,0.133422,14.99,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,97.2,-1.960095,8.448868,1.587236,6.725092,0.000000,0.000000,0.625588,0.000000,0.000000,0.000000
20,2016-02-14,5686.591837,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,97.2,-3.446808,5.039228,4.526593,0.000000,0.000000,0.000000,0.512635,0.000000,0.000000,0.000000
23,2016-02-21,5867.561224,138936953,7,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,97.2,-2.556119,5.649348,4.785689,0.000000,0.000000,0.000000,0.863659,0.000000,0.000000,0.000000
26,2016-02-28,6017.255102,138936953,2,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,97.2,-3.862833,5.048539,5.010574,0.000000,0.000000,0.000000,0.037965,0.000000,0.000000,0.000000
29,2016-03-06,6209.806122,138936953,4,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,97.2,-3.148453,5.885017,5.314554,0.000000,0.000000,0.000000,0.570463,0.000000,0.000000,0.000000


In [122]:
model_138936953['sum'] = model_138936953['base'] + model_138936953['weekly_avg_disc_base'] + model_138936953['Email_base'] + model_138936953['Flyer_base'] + model_138936953['Paid_Search_base'] + model_138936953['Web_Display_base'] + model_138936953['TV_base'] + model_138936953['Radio_base']

In [123]:
model_138936953['base'] = (model_138936953['base']*model_138936953['weekly_sale_qty'])/model_138936953['sum']

In [124]:
for i in ['weekly_avg_disc', 'Email', 'Flyer', 'Paid_Search', 'Web_Display', 'TV', 'Radio']:
    tmp = i + "_base"
    model_138936953[tmp] = (model_138936953[tmp]*model_138936953['weekly_sale_qty'])/model_138936953['sum']

In [125]:
model_138936953

,tran_wk,seas_index,prod_id,weekly_sale_qty,weekly_avg_disc,weekly_shelf_price,holiday,Flyer,Store_Display,TV,Radio,Paid_Search,Web_Display,Email,max_sales_qty,sales_trfm,pred,base,weekly_avg_disc_base,Email_base,Flyer_base,Paid_Search_base,Web_Display_base,TV_base,Radio_base,sum
1,2016-01-03,5435.051020,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,47733,0.0,0.0,97.2,-3.446808,5.208167,2.412679,0.000000,0.000000,0.000000,0.587321,0.000000,0.000000,0.000000,5.208167
5,2016-01-10,5335.275510,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,29342,0.0,0.0,97.2,-2.721295,4.645517,5.245374,0.000000,0.000000,0.000000,0.754626,0.000000,0.000000,0.000000,4.645517
8,2016-01-17,5262.316327,138936953,6,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,31367,0.0,0.0,97.2,-2.721295,4.584599,5.196325,0.000000,0.000000,0.000000,0.803675,0.000000,0.000000,0.000000,4.584599
9,2016-01-24,5252.102041,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,24650,0.0,0.0,97.2,-3.446808,4.431949,2.679158,0.000000,0.000000,0.000000,0.320842,0.000000,0.000000,0.000000,4.431949
14,2016-01-31,5327.520408,138936953,8,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,3288,0.0,0.0,97.2,-2.411439,4.113170,7.880068,0.000000,0.000000,0.000000,0.119932,0.000000,0.000000,0.000000,4.113170
17,2016-02-07,5516.459184,138936953,12,0.133422,14.99,CARNIVAL,0.0,0.0,0.000000,0.000000,17507,0.0,0.0,97.2,-1.960095,8.448868,2.131015,9.029075,0.000000,0.000000,0.839911,0.000000,0.000000,0.000000,8.937915
20,2016-02-14,5686.591837,138936953,3,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,23529,0.0,0.0,97.2,-3.446808,5.039228,2.694813,0.000000,0.000000,0.000000,0.305187,0.000000,0.000000,0.000000,5.039228
23,2016-02-21,5867.561224,138936953,7,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,36556,0.0,0.0,97.2,-2.556119,5.649348,5.929857,0.000000,0.000000,0.000000,1.070143,0.000000,0.000000,0.000000,5.649348
26,2016-02-28,6017.255102,138936953,2,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,1660,0.0,0.0,97.2,-3.862833,5.048539,1.984960,0.000000,0.000000,0.000000,0.015040,0.000000,0.000000,0.000000,5.048539
29,2016-03-06,6209.806122,138936953,4,0.000000,14.99,0,0.0,0.0,0.000000,0.000000,22561,0.0,0.0,97.2,-3.148453,5.885017,3.612261,0.000000,0.000000,0.000000,0.387739,0.000000,0.000000,0.000000,5.885017


In [138]:
model_138936951.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936951.csv",index=False)
model_138936952.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936952.csv",index=False)
model_138936953.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936953.csv",index=False)

In [134]:
model_51 = model_138936951.loc[:,["tran_wk","base","weekly_avg_disc_base","Email_base","Flyer_base","Paid_Search_base","Web_Display_base","TV_base","Radio_base"]].copy()
model_51.columns = ["tran_wk","base","weekly_avg_disc","Email","Flyer","Paid_Search","Web_Display","TV","Radio"]
model_51 = model_51.set_index("tran_wk").stack().reset_index() 
model_51.columns = ["tran_wk","Source","weekly_sale_qty"]

In [135]:
model_52 = model_138936952.loc[:,["tran_wk","base","weekly_avg_disc_base","Email_base","Flyer_base","Paid_Search_base","Web_Display_base","TV_base","Radio_base"]].copy()
model_52.columns = ["tran_wk","base","weekly_avg_disc","Email","Flyer","Paid_Search","Web_Display","TV","Radio"]
model_52 = model_52.set_index("tran_wk").stack().reset_index() 
model_52.columns = ["tran_wk","Source","weekly_sale_qty"]

In [136]:
model_53 = model_138936953.loc[:,["tran_wk","base","weekly_avg_disc_base","Email_base","Flyer_base","Paid_Search_base","Web_Display_base","TV_base","Radio_base"]].copy()
model_53.columns = ["tran_wk","base","weekly_avg_disc","Email","Flyer","Paid_Search","Web_Display","TV","Radio"]
model_53 = model_53.set_index("tran_wk").stack().reset_index() 
model_53.columns = ["tran_wk","Source","weekly_sale_qty"]

In [137]:
model_51.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936951.csv",index=False)
model_52.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936952.csv",index=False)
model_53.to_csv("C:/Users/10331/OneDrive/Desktop/model_138936953.csv",index=False)